In [2]:
import os

# đọc video từ thư mục video
video_dir = 'video/'
videos = os.listdir(video_dir)
videos = [os.path.join(video_dir, f) for f in videos]

In [3]:
import cv2

from module.prediction import acceleration, check_coordinate, check_static
from module.pose_landmarker import draw, extract_pose_features

fps = 15.0
sampling_interval = 1.5  # Độ dài của mẫu
prediction_interval = 0.5  # Khoảng cách giữa các mẫu

In [4]:
result = [0, 0, 0, 0]
for video in videos:
    # Đọc video
    cap = cv2.VideoCapture(video)
    print(video)
    cap.set(cv2.CAP_PROP_FPS, fps)
    df = []

    # Kiểm tra xem video có được mở thành công không
    if not cap.isOpened():
        print("Không thể mở video.")
        exit()

    fall = False

    while True:
        # Đọc frame từ webcam
        ret, frame = cap.read()

        # Nếu không đọc được frame thì thoát khỏi vòng lặp
        if not ret:
            break
    
        # Trích xuất các điểm mốc
        landmarks = extract_pose_features(frame)

        # Nếu có điểm mốc thì thêm vào df
        if landmarks is not None:
            df.append(landmarks)

        if len(df) >= sampling_interval * fps:

            # Nếu quá 30% mẫu có tọa độ bằng 0 thì bỏ qua (của tất cả các điểm mốc chứ không phải của mỗi điểm 0)
            if (
                sum([1 for i in df for j in i for k in j if k == -1])
                > len(df) * 13 * 3 * 0.3
            ):
                df = df[int(prediction_interval * fps) :]
                continue

            if acceleration(df, fps, 3) and check_coordinate(df):
                fall = True

            df = df[int(prediction_interval * fps) :]
    
    if video[6] == '1' and fall:
        result[0] += 1
    elif video[6] == '0' and fall:
        result[1] += 1
    elif video[6] == '1' and not fall:
        result[2] += 1
    elif video[6] == '0' and not fall:
        result[3] += 1

video/0 (1).mp4
video/0 (10).mp4
video/0 (11).mp4
video/0 (12).mp4
video/0 (13).mp4
video/0 (14).mp4
video/0 (15).mp4
video/0 (16).mp4
video/0 (17).mp4
video/0 (18).mp4
video/0 (19).mp4
video/0 (2).mp4
video/0 (20).mp4
video/0 (21).mp4
video/0 (22).mp4
video/0 (23).mp4
video/0 (24).mp4
video/0 (25).mp4
video/0 (26).mp4
video/0 (27).mp4
video/0 (28).mp4
video/0 (29).mp4
video/0 (3).mp4
video/0 (30).mp4
video/0 (31).mp4
video/0 (32).mp4
video/0 (33).mp4
video/0 (34).mp4
video/0 (35).mp4
video/0 (36).mp4
video/0 (37).mp4
video/0 (38).mp4
video/0 (39).mp4
video/0 (4).mp4
video/0 (40).mp4
video/0 (41).mp4
video/0 (42).mp4
video/0 (43).mp4
video/0 (44).mp4
video/0 (45).mp4
video/0 (46).mp4
video/0 (47).mp4
video/0 (48).mp4
video/0 (49).mp4
video/0 (5).mp4
video/0 (50).mp4
video/0 (51).mp4
video/0 (52).mp4
video/0 (53).mp4
video/0 (54).mp4
video/0 (55).mp4
video/0 (56).mp4
video/0 (57).mp4
video/0 (58).mp4
video/0 (59).mp4
video/0 (6).mp4
video/0 (60).mp4
video/0 (7).mp4
video/0 (8).mp4
video

In [9]:
print(result)
tf, ff, fn, tn = result
precision = tf / (tf + ff)
recall = tf / (tf + fn)
f1 = 2 * precision * recall / (precision + recall)
print(f'Precision: {precision}') #là độ chính xác trung bình
print(f'Recall: {recall}') #là độ phủ trung bình
print(f'F1: {f1}') #là độ dung hòa trung bình

[15, 0, 2, 60]
Precision: 1.0
Recall: 0.8823529411764706
F1: 0.9375
